## 1. Robot and Map classes


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time 

In [3]:
import sys 
sys.path.append('.\src')

from main import *



Thymio 1: (208.93, 166.0)
Thymio 2: (178.87, 166.0)
Goal_Center 1: (144.67, 70.0)
Thymio 1: (208.6, 165.6)
Thymio 2: (178.27, 165.3)
Goal_Center 1: (146.33, 70.0)
Thymio 1: (208.77, 165.5)
Thymio 2: (178.2, 165.2)
Green Object 1: (241.95, 77.95)
Green Object 2: (106.96, 69.05)
Green Object 3: (69.96, 228.92)
Green Object 4: (255.88, 230.88)
Invalid coordinate format: ('top_right', (241.95, 77.95))
Invalid coordinate format: ('top_left', (106.96, 69.05))
Invalid coordinate format: ('bottom_left', (69.96, 228.92))
Invalid coordinate format: ('bottom_right', (255.88, 230.88))
Goal_Center 1: (145.33, 70.0)
Thymio 1: (208.7, 165.67)
Thymio 2: (178.27, 165.27)
Goal_Center 1: (144.33, 70.0)
Thymio 1: (208.67, 165.67)
Thymio 2: (178.17, 165.33)
Goal_Center 1: (145.33, 70.0)
Thymio 1: (208.6, 165.53)
Thymio 2: (178.33, 165.3)
Green Object 1: (241.95, 77.95)
Green Object 2: (106.96, 68.96)
Green Object 3: (69.96, 228.96)
Green Object 4: (255.95, 230.95)
Invalid coordinate format: ('top_right', (

KeyboardInterrupt: 

In [3]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

NameError: name 'thymio_markers' is not defined

: 